In [ ]:
from geopy.distance import geodesic
from operator import attrgetter

class Material:
    def __init__(self, id, volume, start_loc, end_loc):
        self.id = id
        self.volume = volume
        self.start_loc = start_loc
        self.end_loc = end_loc
        self.fitted = False

    def __repr__(self):
        return f"{self.id}, {self.volume}, {self.end_loc}"

class Truck:
    def __init__(self, id, volume):
        self.id = id
        self.volume = volume
        self.remaining_volume = volume
        self.current_loc = None
        self.materials = []

    def __repr__(self):
        return f"{self.id}, {', '.join([m.id for m in self.materials])}"

def get_distance(src, dest):
    return geodesic(src, dest).km

#group the materials for each destination and sort
def fit_materials(materials, trucks, max_distance=1500):
    sorted_materials = sorted(materials, key=lambda x: (x.volume, x.end_loc))
    sorted_trucks = sorted(trucks, key=attrgetter('volume'))

    for material in sorted_materials:
        if material.fitted:
            continue

        fitted = False
        closest_dest = None
        closest_dist = float('inf')

        for truck in sorted_trucks:
            if truck.remaining_volume >= material.volume:
                if truck.current_loc is None:
                    truck.current_loc = material.start_loc

                dist = get_distance(truck.current_loc, material.end_loc)

                if dist < closest_dist:
                    closest_dest = material.end_loc
                    closest_dist = dist

                if closest_dest == material.end_loc and dist <= max_distance:
                    truck.materials.append(material)
                    truck.remaining_volume -= material.volume
                    fitted = True
                    truck.current_loc = material.end_loc
                    material.fitted = True
                    break

        if not fitted:
            continue

        while True:
            next_material = None
            next_closest_dest = None
            next_closest_dist = float('inf')

            for material in sorted_materials:
                if material in truck.materials or material.fitted:
                    continue

                if truck.remaining_volume >= material.volume:
                    dist = get_distance(truck.current_loc, material.end_loc)

                    if dist < next_closest_dist and dist <= max_distance:
                        next_closest_dest = material.end_loc
                        next_closest_dist = dist
                        next_material = material

            if next_material is None:
                break

            truck.materials.append(next_material)
            truck.remaining_volume -= next_material.volume
            truck.current_loc = next_material.end_loc
            next_material.fitted = True

    # add remaining materials to trucks
    for material in sorted_materials:
        if not material.fitted:
            for truck in sorted_trucks:
                if truck.remaining_volume >= material.volume:
                    if truck.current_loc is None:
                        truck.current_loc = material.start_loc

                    truck.materials.append(material)
                    truck.remaining_volume -= material.volume
                    truck.current_loc = material.end_loc
                    material.fitted = True
                    break

    # use remaining trucks to ship materials
    for truck in sorted_trucks:
        for material in sorted_materials:
            if not material.fitted and truck.remaining_volume >= material.volume:
                if truck.current_loc is None:
                    truck.current_loc = material.start_loc

                truck.materials.append(material)
                truck.remaining_volume -= material.volume
                truck.current_loc = material.end_loc
                material.fitted = True

    return trucks

materials = [
    Material("M1", 60.0,"37.9643,91.8381", "47.6062,122.3321"), # Seattle
    Material("M2", 50.0, "37.9643,91.8381", "41.8781,87.6298"), # Chicago
    Material("M3", 55.0, "37.9643,91.8381", "42.3601,71.0589"), # Boston
    Material("M4", 65.0, "37.9643,91.8381", "32.7157,117.1611"), # San Diego
    Material("M5", 70.0, "37.9643,91.8381", "32.7767,96.7970"), # Dallas
    Material("M6", 60.0, "37.9643,91.8381", "46.8797,110.3626"), # Montana
    Material("M7", 55.0, "37.9643,91.8381","39.0119, 98.4842"), #Kansas
    Material("M8", 60.0, "37.9643,91.8381","39.9526,75.1652"), #Philly
    Material("M9", 50.0, "37.9643,91.8381", "32.1656,82.9001") #Georgia

  ]

#Input list for trucks

trucks = [Truck("T1",250), Truck("T2",350.0), Truck("T3",250.0), Truck("T4",600.0), Truck("T5",500.0), Truck("T6",800.0)]

result = fit_materials(materials, trucks)
for truck in result:
    for material in truck.materials:
        print(f"Truck {truck.id} will carry {material.id} to {material.end_loc}")



Truck T1 will carry M9 to 32.1656,82.9001
Truck T1 will carry M8 to 39.9526,75.1652
Truck T1 will carry M3 to 42.3601,71.0589
Truck T1 will carry M2 to 41.8781,87.6298
Truck T2 will carry M4 to 32.7157,117.1611
Truck T3 will carry M7 to 39.0119, 98.4842
Truck T3 will carry M5 to 32.7767,96.7970
Truck T3 will carry M6 to 46.8797,110.3626
Truck T3 will carry M1 to 47.6062,122.3321
